In [1]:
!pip install openai langchain tiktoken langchain-openai langchain-community -q


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


## Если используете ключ от OpenAI, запустите эту ячейку 👇


In [15]:
from langchain_openai import ChatOpenAI
import os
from getpass import getpass


# os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0)

## Если используете ключ из курса, запустите эти ячейки 👇


In [1]:
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

--2025-04-30 22:42:01--  https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11316 (11K) [text/plain]
Saving to: ‘utils.py.1’

utils.py.1          100%[===================>]  11.05K  --.-KB/s    in 0.02s   

2025-04-30 22:42:03 (540 KB/s) - ‘utils.py.1’ saved [11316/11316]



In [50]:
from utils import ChatOpenAI
from getpass import getpass

#course_api_key= "Введите ваш API ключ с курса"
course_api_key = getpass(prompt='Введите API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

## Задание 2.1.8 LLM — сила 💪, ML — могила 🧟

In [51]:
import pandas as pd
from tqdm import tqdm

In [9]:
# df = pd.read_csv('https://stepik.org/media/attachments/lesson/1084297/submission100lines.csv')

In [ ]:
# df = pd.read_csv('https://stepik.org/media/attachments/lesson/1110806/100_crypto_messages.csv')

In [31]:
import pandas as pd

df = pd.read_csv('https://stepik.org/media/attachments/lesson/1110806/vacancies_messages_50.csv')

In [33]:
df.head()

,text_id,text
0,9,#вакансия #vacancy #Python #удаленка #flask #r...
1,31,#ВАКАНСИЯ #Системный_аналитик #РФ\n \nАКЦИОНЕ...
2,28,#вакансия #vacancy #job #senior #data #DB #dat...
3,49,#вакансия #fulltime #remote \n\n🔎 Ищем Руковод...
4,18,#vacancy #job #analyst #travel #sirenatravel #...


In [52]:
from langchain import FewShotPromptTemplate


In [12]:
# Напишите шаблон промпта, указав инструкцию, вопрос и выходной индикатор
prompt_template = """тебе будет передан текст сообщения. Очень вероятно, что он будет содеражть тематику криптовалют. Опередели по содерданию является ли текст спамом

 

Context: {text_input}

 

Question: Это спам?

 

Answer: Если спам верни 1, если нет - верни 0. Ничего кроме цифр

"""

In [53]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [39]:
template = """
Из следующего текста извлеки информацию:

job_title: Как указано в описании вакансии, на том же языке. Проверяй полное совпадение, без учета регистра. 
            Если написан грейд, его нужно убрать (например, Senior Python developer -> Python developer, C++ разработчик (middle, senior) ->C++ разработчик)
            Если информация явно не определена, поставь значение None

company: Как указано в описании вакансии, на том же языке. Проверяй полное совпадение, без учета регистра. 
        Указывай только название (не пиши финтех, крупная компания, мобильная игра и т.д.)
        Если информация явно не определена, поставь значение None.
        
salary: Числа пиши без пробелов, не пиши тыс. или к.(умножай в таком случае на 1000), не пиши фикс, плюшки, премии, % от продаж, техника и т.д.
        Не пиши net, gross, на руки и т.д.
        После числа (диапазона чисел) указывай валюту руб. или $ после пробела.
        Если указан диапазон, то пиши его через тире, около тире пробелы не ставь (например, 100к-150к рублей -> 100000-150000 руб.).
        Если указана только нижняя граница (от 2000$ или 100000+руб), то пиши это значение, используя слово от (например, 100к+руб -> от 100000 руб.).
        Если указана только верхняя граница, то пиши это значение, используя слово до (например, до 100к руб -> до 100000 руб.).
        Если зарплата указана за час, то в конце добавляй в час.
        Если информация явно не определена, поставь значение None.

tg: Указывай контакт в телеграмм, используя @. Проверяй полное совпадение, с учетом регистра.
    Если указано несколько контаков, то указывай их через запятую (не забывая про пробел после запятой).
    Если информация явно не определена, поставь значение None.

grade: Возможные значения intern, junior, junior+, middle, middle+, senior, lead.
        Если указано несколько значений, то пиши их через запятую в порядке возрастания.
        Если информация явно не определена, поставь значение None.
        

text: {text}

{format_instructions}
"""

job_title_schema = ResponseSchema(
    name="job_title",
    description="Как указано в описании вакансии, на том же языке. Проверяй полное совпадение, без учета регистра. "
                "Если написан грейд, его нужно убрать (например, Senior Python developer -> Python developer, C++ разработчик (middle, senior) ->C++ разработчик)"
                "Если информация явно не определена, поставь значение None."
)

company_schema = ResponseSchema(
    name="company",
    description="Как указано в описании вакансии, на том же языке. Проверяй полное совпадение, без учета регистра. "
                "Указывай только название (не пиши финтех, крупная компания, мобильная игра и т.д.)"
                "Если информация явно не определена, поставь значение None."
)

salary_schema = ResponseSchema(
    name="salary",
    description="Числа пиши без пробелов, не пиши тыс. или к.(умножай в таком случае на 1000), не пиши фикс, плюшки, премии, % от продаж, техника и т.д."
                "Не пиши net, gross, на руки и т.д."
                "После числа (диапазона чисел) указывай валюту руб. или $ после пробела."
                "Если указан диапазон, то пиши его через тире, около тире пробелы не ставь (например, 100к-150к рублей -> 100000-150000 руб.)."
                "Если указана только нижняя граница (от 2000$ или 100000+руб), то пиши это значение, используя слово от (например, 100к+руб -> от 100000 руб.)."
                "Если указана только верхняя граница, то пиши это значение, используя слово до (например, до 100к руб -> до 100000 руб.)."
                "Если зарплата указана за час, то в конце добавляй в час."
                "Если информация явно не определена, поставь значение None."
)

tg_schema = ResponseSchema(
    name="tg",
    description="Указывай контакт в телеграмм, используя @. Проверяй полное совпадение, с учетом регистра. "
                "Если указано несколько контаков, то указывай их через запятую (не забывая про пробел после запятой)."
                "Если информация явно не определена, поставь значение None."
)

grade_schema = ResponseSchema(
    name="grade",
    description="Возможные значения intern, junior, junior+, middle, middle+, senior, lead. "
                "Если указано несколько значений, то пиши их через запятую в порядке возрастания."
                "Если информация явно не определена, поставь значение None."
)

response_schemas = [
    job_title_schema,
    company_schema,
    salary_schema,
    tg_schema,
    grade_schema
]

In [40]:
amount_list = [] # Список, где будем хранить ответы модели

for text_input in tqdm(df['text']):
    prompt = prompt_template.format(text_input=text_input) # Добавляем сообщение в промпт
    amount = llm.invoke(prompt).content # Ответ модели
    amount_list.append(amount) # Добавляем ответ в список
    # break # Для отладки. Уберите, когда убедитесь, что на одном примере работает

100%|██████████| 50/50 [03:37<00:00,  4.36s/it]


In [41]:
# перед записью ответа в датафрейм, проверьте какого типа ответы
type(amount_list[0])

str

In [42]:
amount_list

['0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0']

In [19]:
# Чтобы система зачла ответы на Степике, надо перевести их в числа
# Если не знаете как, спросите у ChatGPT)

In [43]:
df

,text_id,text
0,9,#вакансия #vacancy #Python #удаленка #flask #r...
1,31,#ВАКАНСИЯ #Системный_аналитик #РФ\n \nАКЦИОНЕ...
2,28,#вакансия #vacancy #job #senior #data #DB #dat...
3,49,#вакансия #fulltime #remote \n\n🔎 Ищем Руковод...
4,18,#vacancy #job #analyst #travel #sirenatravel #...
5,48,#вакансия #fulltime #remote #lookfor #devops\n...
6,41,#вакансия #senior #middle+ #lead #удаленка #оф...
7,6,#job #python #django #javascript #react #fulls...
8,17,#вакансия #vacancy #middle #senior #remote #уд...
9,4,Вакансия: Разработчик (Vue.js / Golang) для на...


In [44]:
df['amount'] = amount_list # Создаём новый столбец из ответов модели

In [45]:
import polars as pl

In [46]:
df = pl.from_pandas(df)

In [47]:
df.with_columns(pl.col('amount').alias('is_spam'))

text_id,text,amount,is_spam
i64,str,str,str
9,"""#вакансия #vacancy #Python #уд…","""0""","""0"""
31,"""#ВАКАНСИЯ #Системный_аналитик…","""0""","""0"""
28,"""#вакансия #vacancy #job #senio…","""0""","""0"""
49,"""#вакансия #fulltime #remote …","""0""","""0"""
18,"""#vacancy #job #analyst #travel…","""0""","""0"""
…,…,…,…
36,"""#вакансия Middle+/Senior Back…","""0""","""0"""
10,"""#вакансия Всем привет! Ищем…","""0""","""0"""
11,"""#вакансия #vacancy #DWH #remot…","""0""","""0"""


In [30]:
df.rename({'amount': 'is_spam'}).with_columns(pl.col('is_spam').cast(pl.Int8)).write_csv('submission.csv')

In [13]:
df.with_columns(pl.col('amount').cast(pl.Int64)).write_csv('2_1_8_solution.csv')

In [18]:
df

text_id,text,amount,is_spam
i64,str,str,str
143,"""Удаленная работа. От нас: Зп о…","""0""","""0"""
179,"""Что вы думаете о концепции дец…","""0""","""0"""
147,"""Удалённый заработок. Время 1-2…","""1""","""1"""
157,"""Меня всегда интересовал вопрос…","""0""","""0"""
168,"""Я слышал, что некоторые банки …","""0""","""0"""
…,…,…,…
209,"""Работа в сфере криптовалютной …","""0""","""0"""
200,"""Какие криптовалюты вы видите в…","""0""","""0"""
229,"""Исследование и анализ рынка кр…","""0""","""0"""


In [ ]:
df.to_csv('2_1_8_solution.csv', index=False) # Сохраняем файл, отправляем на Stepik, получаем баллы :)

In [54]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/1110806/vacancies_messages_50.csv')

# Промпт-шаблон
prompt_template = """
Из следующего текста извлеки информацию:

job_title: Как указано в описании вакансии, на том же языке. Проверяй полное совпадение, без учета регистра. Если написан грейд, его нужно убрать (например, Senior Python developer -> Python developer, C++ разработчик (middle, senior) -> C++ разработчик). Если информация явно не определена, поставь значение None.

company: Как указано в описании вакансии, на том же языке. Проверяй полное совпадение, без учета регистра. Указывай только название (не пиши финтех, крупная компания, мобильная игра и т.д.). Если информация явно не определена, поставь значение None.

salary: Числа пиши без пробелов, не пиши тыс. или к (умножай в таком случае на 1000), не пиши фикс, плюшки, премии, % от продаж, техника и т.д. Не пиши net, gross, на руки и т.д. После числа (диапазона чисел) указывай валюту руб. или $ после пробела. Если указан диапазон, то пиши его через тире, около тире пробелы не ставь (например, 100к-150к рублей -> 100000-150000 руб.). Если указана только нижняя граница (от 2000$ или 100000+руб), то пиши это значение, используя слово от (например, 100к+руб -> от 100000 руб.). Если указана только верхняя граница, то пиши это значение, используя слово до (например, до 100к руб -> до 100000 руб.). Если зарплата указана за час, то в конце добавляй "в час". Если информация явно не определена, поставь значение None.

tg: Указывай контакт в телеграмм, используя @. Проверяй полное совпадение, с учетом регистра. Если указано несколько контактов, то указывай их через запятую (не забывая про пробел после запятой). Если информация явно не определена, поставь значение None.

grade: Возможные значения intern, junior, junior+, middle, middle+, senior, lead. Если указано несколько значений, то пиши их через запятую в порядке возрастания. Если информация явно не определена, поставь значение None.

text: {text}

Ответ верни в формате JSON с ключами: job_title, company, salary, tg, grade.
"""

In [60]:
from utils import ChatOpenAI
from getpass import getpass

course_api_key = getpass(prompt='Введите API ключ')
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

In [61]:
import os
from getpass import getpass

os.environ['OPENAI_API_KEY'] = getpass('Введите ваш OpenAI API ключ: ')
llm = ChatOpenAI(temperature=0.0)

TypeError: ChatOpenAI.__init__() missing 1 required positional argument: 'course_api_key'

In [63]:
# Используем уже инициализированную модель llm из предыдущих ячеек
# llm уже инициализирован, не нужно создавать еще раз

results = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    prompt = prompt_template.format(text=row['text'])
    response = llm.invoke(prompt).content
    # Парсим JSON-ответ (можно использовать json.loads, если ответ корректный)
    try:
        data = eval(response)  # Лучше использовать json.loads, если ответ всегда валидный JSON
    except Exception:
        data = {"job_title": None, "company": None, "salary": None, "tg": None, "grade": None}
    results.append(data)

# Добавляем новые столбцы в DataFrame
df_out = df.copy()
for key in ["job_title", "company", "salary", "tg", "grade"]:
    df_out[key] = [res.get(key) for res in results]

# Сохраняем результат
df_out.to_csv("vacancies_extracted.csv", index=False)

100%|██████████| 50/50 [02:24<00:00,  2.88s/it]
